In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
import os
import time
import random
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchsummary import summary

In [3]:
from tqdm import tqdm
from MNISTData import MNISTData
from AutoEncoder import AutoEncoder

In [4]:
ENCODING_SIZE = 10
NUM_QUBITS = 10

In [5]:
dev = qml.device('default.qubit', wires=NUM_QUBITS)

In [6]:
from pennylane.ops import RX, RY, CNOT

# x will be a length ENCODING_SIZE vector that represents
# the encoding of a MNIST image
# thetas is of size 2 * NUM_QUBITS
@qml.qnode(dev)
def circuit(x, thetas):
    for i in range(ENCODING_SIZE):
        RX(x[i], wires=i)
    for i in range(NUM_QUBITS - 1):
        CNOT(wires=[i, i+1])
    for i in range(NUM_QUBITS):
        RX(thetas[i], wires=i)
    for i in range(NUM_QUBITS, 2 * NUM_QUBITS):
        RY(thetas[i], wires=(i - NUM_QUBITS))
    return tuple(qml.expval.PauliZ(wires=i) for i in range(NUM_QUBITS))

In [7]:
# example input
print(circuit([np.pi/5]*(ENCODING_SIZE), [np.pi]*(2 * NUM_QUBITS)))

[0.80901699 0.6545085  0.5295085  0.42838137 0.34656781 0.28037925
 0.22683158 0.1835106  0.14846319 0.12010925]


In [8]:
def cost(X, actual_labels, thetas):
    b = X.shape[0]
    yhats = []
    for i in range(b):
        yhat = circuit(X[i], thetas)
        yhats.append(yhat)
    st = np.stack(yhats)
    actual_class_vals = st[range(b), actual_labels]
    shifted = st - np.max(st, axis=1)[:, np.newaxis]
    the_sum = np.log(np.sum(np.exp(shifted), axis=1))
    return np.mean(-actual_class_vals + the_sum)

At this point, we have to get our (encoded) images so that we may actually start training

In [9]:
data = MNISTData(batch_size=4)
train_loader = data.get_train_loader()
test_loader = data.get_test_loader()

In [10]:
load_from_stem = "./autoencoder_models/1558731906/"
load_model_from = os.path.join(load_from_stem, "ae.pt")
load_layers_from = os.path.join(load_from_stem, "layer_sizes.pkl")

In [11]:
with open(load_layers_from, 'rb') as f:
    layer_sizes = pickle.load(f)
layer_sizes = layer_sizes[1:]
ae = AutoEncoder(layer_sizes)
ae.load_state_dict(torch.load(load_model_from))
encoder = ae.encoder
for child in encoder.children():
    for param in child.parameters():
        param.requires_grad = False

In [12]:
summary(encoder, input_size=(28 * 28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]         200,960
              ReLU-2                  [-1, 256]               0
            Linear-3                   [-1, 64]          16,448
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 10]             650
              ReLU-6                   [-1, 10]               0
Total params: 218,058
Trainable params: 0
Non-trainable params: 218,058
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.83
Estimated Total Size (MB): 0.84
----------------------------------------------------------------


In [13]:
thetas = 2 * np.random.rand(2 * NUM_QUBITS) - 1
print(thetas)
print(thetas.dtype)

[ 0.96619343 -0.14789545  0.36387268  0.92188894 -0.85160081  0.85258638
 -0.28277243 -0.46211122 -0.81660402 -0.12947143 -0.8954551   0.56925904
 -0.6602414   0.76241819  0.59891522  0.99119002  0.85932832  0.62668308
 -0.53348369 -0.63579895]
float64


In [14]:
# this is the transformation that you need to do in order
# to pass it to the encoder. it is (1, -1) since the first number
# is the batch size, which in our case is 1
enums = enumerate(train_loader, 0)
i, data = next(enums)
inputs, labels = data
print(inputs.size())
print(len(labels))
print(encoder(inputs[0].view(1, -1)))
print(encoder(inputs[0].view(1, -1)).size())

torch.Size([4, 1, 28, 28])
4
tensor([[44.7678, 24.3530,  0.0000, 13.3956, 32.0997,  8.4641,  9.2724, 16.5090,
         28.4740,  0.0000]])
torch.Size([1, 10])


In [15]:
X = encoder(inputs.view(len(labels), -1))
print(X.size())
print(labels.size())
start = time.time()
qml.grad(cost, argnum=2)(X.numpy(), labels.numpy(), thetas)
print(time.time() - start)

torch.Size([4, 10])
torch.Size([4])
199.52496814727783
